In [1]:

import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


In [2]:
batch_size = 32
seed = 42

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "./data/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed
)

raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    './data/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed
)

raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    './data/test',
    batch_size=batch_size
)


Found 18563 files belonging to 7 classes.
Using 14851 files for training.
Found 18563 files belonging to 7 classes.
Using 3712 files for validation.
Found 1903 files belonging to 7 classes.


In [4]:
max_features = 10000
sequence_length = 250

vectorize_layer = TextVectorization(
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length
)


In [5]:
# Make a text-only dataset (without labels), then call adapt
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)


In [5]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

In [6]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [7]:
embedding_dim = 16

model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(7)])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          160016    
_________________________________________________________________
dropout (Dropout)            (None, None, 16)          0         
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 7)                 119       
Total params: 160,135
Trainable params: 160,135
Non-trainable params: 0
_________________________________________________________________


In [8]:

model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True), 
    optimizer='adam', 
    metrics=['accuracy'])


In [9]:
epochs = 100
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/100
303/303 [==============================] - 15s 48ms/step - loss: 1.9249 - accuracy: 0.2035 - val_loss: 1.8993 - val_accuracy: 0.2102
Epoch 2/100
303/303 [==============================] - 1s 3ms/step - loss: 1.8865 - accuracy: 0.2167 - val_loss: 1.8770 - val_accuracy: 0.2115
Epoch 3/100
303/303 [==============================] - 1s 3ms/step - loss: 1.8591 - accuracy: 0.2273 - val_loss: 1.8372 - val_accuracy: 0.2412
Epoch 4/100
303/303 [==============================] - 1s 3ms/step - loss: 1.8144 - accuracy: 0.2756 - val_loss: 1.7802 - val_accuracy: 0.3147
Epoch 5/100
303/303 [==============================] - 1s 3ms/step - loss: 1.7539 - accuracy: 0.3445 - val_loss: 1.7122 - val_accuracy: 0.3784
Epoch 6/100
303/303 [==============================] - 1s 3ms/step - loss: 1.6839 - accuracy: 0.3959 - val_loss: 1.6419 - val_accuracy: 0.4308
Epoch 7/100
303/303 [==============================] - 1s 3ms/step - loss: 1.6164 - accuracy: 0.4385 - val_loss: 1.5749 - val_accuracy: 0.47

In [10]:
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)


32/32 [==============================] - 4s 96ms/step - loss: 0.7564 - accuracy: 0.7260
Loss:  0.7564383149147034
Accuracy:  0.7260000109672546


In [11]:
export_model = tf.keras.Sequential([
  vectorize_layer,
  model,
  layers.Activation('sigmoid')
])

export_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=False), optimizer="adam", metrics=['accuracy']
)

# Test it with `raw_test_ds`, which yields raw strings
loss, accuracy = export_model.evaluate(raw_test_ds)
print(accuracy)

32/32 [==============================] - 1s 12ms/step - loss: 0.7931 - accuracy: 0.7228
0.7260000109672546


In [28]:


inp = input("> ")
result = export_model.predict([inp])[0]
best_index = 0
best_score = 0
for i in range(len(result)):
    print(f"{raw_train_ds.class_names[i]}: {result[i]}")
    if result[i] > best_score:
        best_score = result[i]
        best_index = i
print("Result: " + raw_train_ds.class_names[best_index])

da: 0.9862661361694336
en: 5.708331585196902e-10
es: 3.177326107106637e-06
ja: 0.000627666711807251
nb: 0.9988806247711182
nn: 0.9837482571601868
sv: 0.5332168936729431
Result: nb
